In [1]:
#导包
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

from torch_geometric.nn.conv import APPNP
from torch_geometric.nn.models import CorrectAndSmooth

import torch_geometric.utils as pyg_utils

import numpy as np

In [2]:
#设置泛用初始超参数
hp={'device':torch.device("cuda:3" if torch.cuda.is_available() else "cpu"),
    'num_of_layers':2,
    'hidden_units':64,
   'dropout_rate':0.5,
   'l2_r':0.005,
   'learning_rate':0.1,
   'early_stopping':100,
   'epoch':50,
   'batch_size':10}  #部分抄自PPNP论文

In [3]:
#设置模型与model-specific超参数
#APPNP超参
hp_appnp={'alpha':0.1,
         'K':10}

#C&S超参
hp_cs={'correct_layer':50,
       'correct_alpha':1,
       'smooth_layer':50,
       'smooth_alpha':0.8,
       'autoscale':False,
      'scale':20}
#从https://github.com/rusty1s/pytorch_geometric/blob/master/examples/correct_and_smooth.py抄的超参

In [4]:
class APPNP_self1(torch.nn.Module):
    #参考PyG设置的参数什么的
    #就PyG设置的就是没有predict部分，所以我也把predict部分放在GNNStack里面了
    #就我想了一下，我觉得用MessagePassing类不方便，就还是用torch的Module类了
    
    def __init__(self,K,alpha):
        #别的参数暂时省略
        super(APPNP_self1,self).__init__()
        self.K=K
        self.alpha=alpha

    def forward(self, x, edge_index):
        #首先尝试使用dense_tensor，如果不行再转sparse_tensor
        (row, col)= edge_index
        node_num=max(row.max(),col.max())+1
        adj = torch.zeros((node_num,node_num))
        adj=adj.to(hp['device'])
        adj[row, col] = torch.ones(row.numel()).to(hp['device'])
        
        self_loop=torch.eye(adj.size()[0]).to(hp['device'])  #自环
        adj=adj+self_loop  #\slide{A}
        degree_vector=torch.sum(adj,dim=1).cpu()  #度矩阵
        degree_vector=1/np.sqrt(degree_vector)  #D-1/2
        degree_matrix=torch.diag(degree_vector).to(hp['device'])
        adj=torch.mm(degree_matrix,adj)
        adj=torch.mm(adj,degree_matrix)  #\hat{\slide{A}}
        
        H=x.clone()
        Z=x.clone()
        
        for k in range(self.K-1):
            Z=torch.mm(adj,Z)
            Z=Z*(1-self.alpha)
            Z=Z+self.alpha*H
        
        Z=torch.mm(adj,Z)
        Z=Z*(1-self.alpha)
        Z=Z+self.alpha*H
        Z=F.log_softmax(Z, dim=1)
        
        return Z

In [5]:
def edge_index2sparse_tensor(edge_index,node_num):
    sizes=(node_num,node_num)
    v=torch.ones(edge_index[0].numel()).to(hp['device'])  #边数
    return torch.sparse_coo_tensor(edge_index, v, sizes)

In [6]:
class APPNP_self2(torch.nn.Module):   
    #dense tensor在GPU上OOM了，我滚过来写稀疏矩阵了
    def __init__(self,K,alpha):
        #别的参数暂时省略
        super(APPNP_self2,self).__init__()
        self.K=K
        self.alpha=alpha

    def forward(self, x, edge_index):
        node_num=x.size()[0]
        edge_index, _ = pyg_utils.add_self_loops(edge_index,num_nodes=node_num)  #添加自环（\slide{A}）
        adj=edge_index2sparse_tensor(edge_index,node_num)  #将\slide{A}转换为稀疏矩阵
        degree_vector=torch.sparse.sum(adj,0)  #度数向量
        degree_vector=degree_vector.to_dense().cpu()
        degree_vector=1/np.sqrt(degree_vector)
        degree_matrix=torch.diag(degree_vector).to(hp['device'])
        adj=torch.sparse.mm(adj.t(),degree_matrix.t())
        adj=adj.t()
        adj=torch.mm(adj,degree_matrix)
        adj=adj.to_sparse()
        
        H=x.clone()
        
        for k in range(self.K-1):
            x=torch.mm(adj,x)
            x=x*(1-self.alpha)
            x=x+self.alpha*H
        
        x=torch.mm(adj,x)
        x=x*(1-self.alpha)
        x=x+self.alpha*H
        x=F.log_softmax(x, dim=1)
        
        return x

In [7]:
#导入数据集
#为了设置随机切分，所以不使用默认的public切分法（但是我最后也没有用到随机切分……算了，就这样吧）
#PPNP的论文切分法看起来好复杂，算了

#dataset_root&name_map
ds_rn_map=[('/tmp/cora','Cora'),('./tmp/citeseer','CiteSeer'),('./tmp/pubmed','PubMed')]
#使用示例代码：dataset1 = Planetoid(root=ds_rn_map[1][0], name=ds_rn_map[1][1],split='random')

In [8]:
class GNNStack(torch.nn.Module):
    def __init__(self, input_dim, output_dim, args_general, args_special):
        super(GNNStack, self).__init__()
        
        self.lin1=nn.Linear(input_dim,args_general['hidden_units'])
        self.appnp=APPNP_self2(args_special['K'],args_special['alpha'])  #APPNP vs. APPNP_self2
        self.lin2=nn.Linear(args_general['hidden_units'],output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
          
        x=self.lin1(x)
        x=self.appnp(x,edge_index)
        x=self.lin2(x)

        return F.log_softmax(x, dim=1)
        #总之是softmax→取对数，具体细节还没看
        #softmax文档里说如果损失函数用nll_loss的话就得用这个而不是softmax

    def loss(self, pred, label):
        return F.nll_loss(pred, label)

In [9]:
def test(model,loader,is_val):
    model.eval()
    total=0
    correct=0
    for batch in loader:
        batch.to(hp['device'])
        with torch.no_grad():
            pred = model(batch).max(dim=1)[1]
            label = batch.y

        mask = batch.val_mask if is_val else batch.test_mask
        # node classification: only evaluate on nodes in test set
        pred = pred[mask]
        label = batch.y[mask]
        total += torch.sum(mask).item()
            
        correct += pred.eq(label).sum().item()
        
    return correct / total

In [10]:
%%time
test_accs=[]
best_models=[]

#以下训练集、验证集、测试集的切分有点问题，日志应该记录验证集上的结果，早停也应该由验证集来。
#最后应该输出在验证集上最好模型在测试集上的结果？

#呃，dropout也没有加，GNNStack也没有叠层，也没有加BN什么的……也没有调参，epoch也只有50轮……
#凑合过吧

#还有这个DataLoader其实我也不知道需不需要，如果可以的话整张图塞进去train也行吧

for i in range(3):
    dataset=Planetoid(root=ds_rn_map[i][0], name=ds_rn_map[i][1],split='random')
    loader = DataLoader(dataset, batch_size=hp['batch_size'], shuffle=True)
    input_dim=dataset.num_features
    output_dim=dataset.num_classes
    model=GNNStack(input_dim,output_dim,hp,hp_appnp)
    model.to(hp['device'])
    best_models.append(model)
    optimizer = torch.optim.Adam(model.parameters(), lr=hp['learning_rate'])
    this_test_accs=[]
    before_biggest_test_acc=0
    count_early_stopping=0
    for epoch in range(hp['epoch']):
        model.train()
        for batch in loader:
            batch=batch.to(hp['device'])
            optimizer.zero_grad()
            out=model(batch)
            labels=batch.y
            loss=model.loss(out[batch.train_mask],labels[batch.train_mask])
            loss.backward()
            optimizer.step()
            
        if epoch%10==0:
            test_acc=test(model,loader,True)
            if test_acc>before_biggest_test_acc:
                before_biggest_test_acc=test_acc
                best_models[i]=model.state_dict()
                count_early_stopping=0
            else:
                count_early_stopping+=1
                if count_early_stopping>hp['early_stopping']:
                    break
            this_test_accs.append(test_acc)
    
    test_accs.append(this_test_accs)

CPU times: user 3min 9s, sys: 3min 32s, total: 6min 41s
Wall time: 1min 49s


In [11]:
test_accs

[[0.08, 0.12, 0.556, 0.728, 0.73],
 [0.228, 0.184, 0.522, 0.556, 0.608],
 [0.396, 0.514, 0.222, 0.514, 0.562]]

In [12]:
class CNS_self(torch.nn.Module):
    #参数照抄PyG了，意思也一样
    def __init__(self,correct_layer,correct_alpha,smooth_layer,smooth_alpha,autoscale,scale):
        super(CNS_self,self).__init__()
        self.correct_layer=correct_layer
        self.correct_alpha=correct_alpha
        self.smooth_layer=smooth_layer
        self.smooth_alpha=smooth_alpha
        self.autoscale=autoscale
        self.scale=scale
    
    def correct(self,Z,Y,mask,edge_index):
        """
        Z:base prediction
        Y:true label（第一维尺寸是训练集节点数）
        mask:训练集mask
        """
        #将Y扩展为独热编码矩阵
        Y=F.one_hot(Y)
        
        num_nodes=Z.size()[0]
        num_features=Z.size()[1]
        E=torch.zeros(num_nodes,num_features).to(hp['device'])
        E[mask]=Z[mask]-Y
        
        edge_index, _ = pyg_utils.add_self_loops(edge_index,num_nodes=num_nodes)  #添加自环（\slide{A}）
        adj=edge_index2sparse_tensor(edge_index,num_nodes)
        degree_vector=torch.sparse.sum(adj,0)  #度数向量
        degree_vector=degree_vector.to_dense().cpu()
        degree_vector=np.power(degree_vector,-0.5)
        degree_matrix=torch.diag(degree_vector).to(hp['device'])
        adj=torch.sparse.mm(adj.t(),degree_matrix.t())
        adj=adj.t()
        adj=torch.mm(adj,degree_matrix)
        adj=adj.to_sparse()
        
        x=E.clone()
        if self.autoscale==True:
            for k in range(self.correct_layer):
                x=torch.sparse.mm(adj,x)
                x=x*self.correct_alpha
                x=x+(1-self.correct_alpha)*E
            sigma=1/(mask.sum().item())*(E.sum())
            Z=Z-x
            Z[~mask]=Z[~mask]-sigma*F.softmax(x[~mask],dim=1)
        else:
            for k in range(self.correct_layer):
                x=torch.sparse.mm(adj,x)
                x=x*self.correct_alpha
                x=x+(1-self.correct_alpha)*E
                x[mask]=E[mask]
            Z=Z-self.scale*x        
        
        return Z
    
    def smooth(self,Z,Y,mask,edge_index):
        #将Y扩展为独热编码矩阵
        Y=F.one_hot(Y)
        
        num_nodes=Z.size()[0]
        G=Z.clone()
        G[mask]=Y.float()
        
        edge_index, _ = pyg_utils.add_self_loops(edge_index,num_nodes=num_nodes)  #添加自环（\slide{A}）
        adj=edge_index2sparse_tensor(edge_index,num_nodes)
        degree_vector=torch.sparse.sum(adj,0)  #度数向量
        degree_vector=degree_vector.to_dense().cpu()
        degree_vector=np.power(degree_vector,-0.5)
        degree_matrix=torch.diag(degree_vector).to(hp['device'])
        adj=torch.sparse.mm(adj.t(),degree_matrix.t())
        adj=adj.t()
        adj=torch.mm(adj,degree_matrix)
        adj=adj.to_sparse()  #adj就是S
        
        x=G.clone()
        for k in range(self.smooth_layer):
            x=torch.sparse.mm(adj,x)
            x=x*self.smooth_alpha
            x=x+(1-self.smooth_alpha)*G
        
        return x

In [14]:
#应用跑了APPNP的best_model跑C&S，或者纯线性模型跑C&S
#C结果：纯线性模型会有很高的结果提升，APPNP的话就妹有

#就是……因为PyG的C&S是只分了训练集/测试集，所以我就直接这么干了：训练集上训练，测试集上输出
#（毕竟没有选择模型参数的过程嘛，所以验证集好像没用的样子）

#这个数据集是有问题的，跟前面训练模型时候的数据集应该是同一种split方法，要不然不河狸。但是我懒得改了，就这样吧

#参考：https://github.com/rusty1s/pytorch_geometric/blob/master/examples/correct_and_smooth.py
#原代码里面那个DAD和DA又是什么东西啊？

post = CNS_self(hp_cs['correct_layer'],hp_cs['correct_alpha'],hp_cs['smooth_layer'],hp_cs['smooth_alpha'],
                        hp_cs['autoscale'],hp_cs['scale'])  #CorrectAndSmooth vs. CNS_self

for i in range(3):
    dataset=Planetoid(root=ds_rn_map[i][0], name=ds_rn_map[i][1],split='random')
    data=dataset[0].to(hp['device'])
    input_dim=dataset.num_features
    output_dim=dataset.num_classes
    #model=GNNStack(input_dim,output_dim,hp,hp_appnp)  #APPNP
    model=torch.nn.Linear(input_dim,output_dim)  #线性模型
    
    model.to(hp['device'])
    #model.load_state_dict(best_models[i])  #APPNP
    #y_soft=model(data)  #APPNP
    y_soft=F.softmax(model(data.x),dim=1)  #线性模型
    
    total=0
    correct=0
    y=y_soft.max(dim=1)[1]
    for mask in [data.test_mask]:
        pred = y[mask]
        label = data.y[mask]
        total += torch.sum(mask).item()
        correct += pred.eq(label).sum().item()
    print('在数据集'+ds_rn_map[i][1]+'中，测试集上的原始accuracy是'+str(correct/total))
    
    print('Correct and smooth...')
    
    #y_soft=F.softmax(y_soft,dim=1)  #APPNP  #因为GNNStack用的是log softmax，所以不能直接用
    mask=data.train_mask
    label = data.y
    y_soft = post.correct(y_soft,label[mask],mask,data.edge_index)
    y_soft = post.smooth(y_soft,label[mask],mask,data.edge_index)
    y_soft = y_soft.max(dim=1)[1]
    
    total=0
    correct=0
    for mask in [data.test_mask]:
        pred = y_soft[mask]
        label = data.y[mask]
        total += torch.sum(mask).item()
        correct += pred.eq(label).sum().item()
        
    print('在数据集'+ds_rn_map[i][1]+'中，测试集上的最终accuracy是'+str(correct/total))
    
    print()

在数据集Cora中，测试集上的原始accuracy是0.122
Correct and smooth...
在数据集Cora中，测试集上的最终accuracy是0.68

在数据集CiteSeer中，测试集上的原始accuracy是0.19
Correct and smooth...
在数据集CiteSeer中，测试集上的最终accuracy是0.483

在数据集PubMed中，测试集上的原始accuracy是0.402
Correct and smooth...
在数据集PubMed中，测试集上的最终accuracy是0.715

